In [7]:
import cv2
import numpy as np
from PIL import Image
import os
from sklearn.cluster import KMeans

# Загрузка изображения
image_path = ('blood_analysis_dataset_540/processed/5256043418088498597.jpg')
image = Image.open(image_path)

# Вычисление новых размеров с сохранением пропорций
width = 900
ratio = width / float(image.size[0])
height = int(float(image.size[1]) * ratio)

# Изменение размера изображения с использованием LANCZOS
resized_image = image.resize((width, height), Image.Resampling.LANCZOS)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_resized{ext}"
new_image_path = os.path.join('resized_data/', new_filename)

# Сохранение измененного изображения
resized_image.save(new_image_path)

# Опционально: показ изображения
resized_image.show()

# Загрузка изображения
ish = new_image_path

# Функция для вычисления угла линии
def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

# Функция для вычисления расстояния между двумя точками
def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

# Функция для вычисления длины линии
def calculate_line_length(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Функция для объединения близких линий
def merge_close_lines(lines, distance_threshold=4, angle_threshold=5):
    """
    Объединяет близко расположенные линии с небольшим угловым отклонением.

    :param lines: Список линий в формате [[x1, y1, x2, y2], ...].
    :param distance_threshold: Максимальное расстояние между линиями для объединения (в пикселях).
    :param angle_threshold: Максимальное угловое отклонение для объединения (в градусах).
    :return: Список объединенных линий.
    """
    merged_lines = []
    used_indices = set()

    for i, line1 in enumerate(lines):
        if i in used_indices:
            continue

        x1, y1, x2, y2 = line1
        angle1 = calculate_angle(x1, y1, x2, y2)
        group = [line1]

        for j, line2 in enumerate(lines[i + 1:], start=i + 1):
            if j in used_indices:
                continue

            x3, y3, x4, y4 = line2
            angle2 = calculate_angle(x3, y3, x4, y4)

            # Проверка углового отклонения
            if abs(angle1 - angle2) > angle_threshold:
                continue

            # Проверка близости линий
            center1 = ((x1 + x2) / 2, (y1 + y2) / 2)
            center2 = ((x3 + x4) / 2, (y3 + y4) / 2)
            distance = calculate_distance(center1, center2)

            if distance > distance_threshold:
                continue

            # Если линии близки и углы схожи, добавляем в группу
            group.append(line2)
            used_indices.add(j)

        # Усреднение линий в группе
        if group:
            avg_line = np.mean(group, axis=0).astype(int)
            merged_lines.append(avg_line)
            used_indices.add(i)

    return merged_lines

# Функция для нахождения точки пересечения двух линий
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    A1 = y2 - y1
    B1 = x1 - x2
    C1 = A1 * x1 + B1 * y1

    A2 = y4 - y3
    B2 = x3 - x4
    C2 = A2 * x3 + B2 * y3

    determinant = A1 * B2 - A2 * B1

    if determinant == 0:
        return None  # Линии параллельны
    else:
        x = (B2 * C1 - B1 * C2) / determinant
        y = (A1 * C2 - A2 * C1) / determinant
        return int(x), int(y)


image = cv2.imread(ish, cv2.IMREAD_COLOR)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение Canny edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Применение Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)

# Преобразование линий в нужный формат
lines = [line[0] for line in lines]

# Объединение близких линий
merged_lines = merge_close_lines(lines, distance_threshold=4, angle_threshold=5)

# Создание копии изображения для рисования линий
output_image = image.copy()

# Получение высоты и ширины изображения
image_height, image_width, _ = image.shape

# Функция для вычисления центра линии
def calculate_center(x1, y1, x2, y2):
    return ((x1 + x2) // 2, (y1 + y2) // 2)

# Словарь для хранения центров линий
line_centers = {}

# Отрисовка всех линий и сохранение их центров
for i, line in enumerate(merged_lines):
    x1, y1, x2, y2 = line
    center = calculate_center(x1, y1, x2, y2)
    line_centers[i] = center

# Фильтрация горизонтальных линий (угол от -40 до 40 градусов)
middle_third_width_start = image_width // 3
middle_third_width_end = 2 * image_width // 3
min_horizontal_line_length = image_width / 4

# Фильтр исключения линий с центрами ближе 20 пикселей к границам
horizontal_lines = {
    line_id: center for line_id, center in line_centers.items()
    if (middle_third_width_start <= center[0] <= middle_third_width_end) and
       (calculate_line_length(*merged_lines[line_id]) >= min_horizontal_line_length) and
       (-40 <= calculate_angle(*merged_lines[line_id]) <= 40) and
       (20 <= center[0] <= image_width - 20) and  # Фильтр по X
       (25 <= center[1] <= image_height - 20)  # Фильтр по Y
}

# Фильтрация вертикальных линий (угол от 50 до 130 градусов)
middle_third_height_start = image_height // 3
middle_third_height_end = 2 * image_height // 3
min_vertical_line_length = image_height / 9

# Фильтр исключения линий с центрами ближе 20 пикселей к границам
vertical_lines = {
    line_id: center for line_id, center in line_centers.items()
    if (middle_third_height_start <= center[1] <= middle_third_height_end) and
       (calculate_line_length(*merged_lines[line_id]) >= min_vertical_line_length) and
       (50 <= abs(calculate_angle(*merged_lines[line_id])) <= 130) and
       (20 <= center[0] <= image_width - 20) and  # Фильтр по X
       (20 <= center[1] <= image_height - 20)  # Фильтр по Y
}

# Нахождение двух горизонтальных и двух вертикальных линий
if horizontal_lines and vertical_lines:
    # Выбираем две горизонтальные линии (с минимальным и максимальным Y)
    min_y_line = min(horizontal_lines.items(), key=lambda item: item[1][1])
    max_y_line = max(horizontal_lines.items(), key=lambda item: item[1][1])

    # Выбираем две вертикальные линии (с минимальным и максимальным X)
    min_x_line = min(vertical_lines.items(), key=lambda item: item[1][0])
    max_x_line = max(vertical_lines.items(), key=lambda item: item[1][0])

    # Получаем координаты линий
    h_line1 = merged_lines[min_y_line[0]]
    h_line2 = merged_lines[max_y_line[0]]
    v_line1 = merged_lines[min_x_line[0]]
    v_line2 = merged_lines[max_x_line[0]]

    # Находим точки пересечения
    intersection_A = line_intersection(h_line1, v_line1)  # Левая верхняя
    intersection_B = line_intersection(h_line2, v_line1)  # Левая нижняя
    intersection_C = line_intersection(h_line1, v_line2)  # Правая верхняя
    intersection_D = line_intersection(h_line2, v_line2)  # Правая нижняя

    # Проверка, что точки пересечения найдены
    if intersection_A and intersection_B and intersection_C and intersection_D:
        # Увеличение изображения
        padding = 0  # Отступ для увеличения изображения
        output_image = cv2.copyMakeBorder(output_image, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=(0, 0, 0))

        # Смещение координат точек пересечения
        intersection_A = (intersection_A[0] + padding, intersection_A[1] + padding)
        intersection_B = (intersection_B[0] + padding, intersection_B[1] + padding)
        intersection_C = (intersection_C[0] + padding, intersection_C[1] + padding)
        intersection_D = (intersection_D[0] + padding, intersection_D[1] + padding)

        # Продление линий до точек пересечения
        cv2.line(output_image, (intersection_A[0], intersection_A[1]), (intersection_C[0], intersection_C[1]), (0, 255, 127), 2)  # Верхняя горизонтальная
        cv2.line(output_image, (intersection_B[0], intersection_B[1]), (intersection_D[0], intersection_D[1]), (0, 255, 127), 2)  # Нижняя горизонтальная
        cv2.line(output_image, (intersection_A[0], intersection_A[1]), (intersection_B[0], intersection_B[1]), (0, 165, 255), 2)  # Левая вертикальная
        cv2.line(output_image, (intersection_C[0], intersection_C[1]), (intersection_D[0], intersection_D[1]), (0, 165, 255), 2)  # Правая вертикальная

        # Подпись точек
        cv2.putText(output_image, "A", (intersection_A[0] - 20, intersection_A[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(output_image, "B", (intersection_B[0] - 20, intersection_B[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(output_image, "C", (intersection_C[0] + 10, intersection_C[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(output_image, "D", (intersection_D[0] + 10, intersection_D[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Сохранение координат точек
        intersections = {
            "A": intersection_A,
            "B": intersection_B,
            "C": intersection_C,
            "D": intersection_D
        }

        # Сохранение координат в файл
        with open("intersections.txt", "w") as file:
            for point, coords in intersections.items():
                file.write(f"{point}: {coords}\n")

print("Координаты точек пересечения:")
for point, coords in intersections.items():
    print(f"{point}: {coords}")
# После вычисления реальных размеров прямоугольника
if intersection_A and intersection_B and intersection_C and intersection_D:
    # Вычисление длин диагоналей на изображении
    diagonal_AB = calculate_distance(intersection_A, intersection_B)
    diagonal_BD = calculate_distance(intersection_B, intersection_D)

    # Отношение AB к стороне BD
    AB_BD_ratio = diagonal_AB / diagonal_BD

    # Предположим, что реальная длина BD известна (например, в метрах)
    real_BD = 1.0  # Например, 1 метр

    # Восстановление истинных размеров прямоугольника
    # Используем пропорции между диагоналями на изображении и реальными размерами
    real_AB = AB_BD_ratio * real_BD

    # Теперь можно вычислить реальные размеры сторон прямоугольника
    # Предположим, что прямоугольник не искажен (т.е. стороны перпендикулярны)
    real_width = real_BD
    real_height = real_AB

    print(f"Истинная ширина прямоугольника: {real_width} метров")
    print(f"Истинная высота прямоугольника: {real_height} метров")

    # Сохранение результатов в файл
    with open("real_dimensions.txt", "w") as file:
        file.write(f"Истинная ширина прямоугольника: {real_width} метров\n")
        file.write(f"Истинная высота прямоугольника: {real_height} метров\n")

    # Вычисление центра изображения
    center_x = image_width // 2
    center_y = image_height // 2

    # Масштабирование реальных размеров для отображения на изображении
    # Предположим, что 1 метр = 100 пикселей (можно изменить в зависимости от масштаба)
    scale_factor = 100  # 1 метр = 100 пикселей
    rect_width_px = int(real_width * scale_factor)
    rect_height_px = int(real_height * scale_factor)

    # Вычисление координат реального прямоугольника
    rect_top_left = (center_x - rect_width_px // 2, center_y - rect_height_px // 2)
    rect_bottom_right = (center_x + rect_width_px // 2, center_y + rect_height_px // 2)

    # Рисование реального прямоугольника сиреневым цветом
    cv2.rectangle(output_image, rect_top_left, rect_bottom_right, (255, 0, 255), 2)  # Сиреневый цвет (BGR)

    # Указание точек A', B', C', D' на реальном прямоугольнике
    rect_A_prime = (rect_top_left[0], rect_top_left[1])
    rect_B_prime = (rect_top_left[0], rect_bottom_right[1])
    rect_C_prime = (rect_bottom_right[0], rect_top_left[1])
    rect_D_prime = (rect_bottom_right[0], rect_bottom_right[1])

    # Рисование точек и подписей
    cv2.circle(output_image, rect_A_prime, 5, (0, 0, 255), -1)  # Точка A'
    cv2.putText(output_image, "A'", (rect_A_prime[0] - 20, rect_A_prime[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.circle(output_image, rect_B_prime, 5, (0, 0, 255), -1)  # Точка B'
    cv2.putText(output_image, "B'", (rect_B_prime[0] - 20, rect_B_prime[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.circle(output_image, rect_C_prime, 5, (0, 0, 255), -1)  # Точка C'
    cv2.putText(output_image, "C'", (rect_C_prime[0] + 10, rect_C_prime[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.circle(output_image, rect_D_prime, 5, (0, 0, 255), -1)  # Точка D'
    cv2.putText(output_image, "D'", (rect_D_prime[0] + 10, rect_D_prime[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Сохранение координат A', B', C', D' в словарь
    rect_prime_coords = {
        "A'": rect_A_prime,
        "B'": rect_B_prime,
        "C'": rect_C_prime,
        "D'": rect_D_prime
    }

    # Сохранение координат в файл
    with open("rect_prime_coords.txt", "w") as file:
        for point, coords in rect_prime_coords.items():
            file.write(f"{point}: {coords}\n")

    # Масштабирование прямоугольника так, чтобы в него вписывался исходный многоугольник ABCD
    # Находим минимальный и максимальный X и Y для исходного многоугольника
    min_x = min(intersection_A[0], intersection_B[0], intersection_C[0], intersection_D[0])
    max_x = max(intersection_A[0], intersection_B[0], intersection_C[0], intersection_D[0])
    min_y = min(intersection_A[1], intersection_B[1], intersection_C[1], intersection_D[1])
    max_y = max(intersection_A[1], intersection_B[1], intersection_C[1], intersection_D[1])

    # Вычисляем ширину и высоту исходного многоугольника
    original_width = max_x - min_x
    original_height = max_y - min_y

    # Вычисляем масштабирующий коэффициент
    scale_x = original_width / rect_width_px
    scale_y = original_height / rect_height_px
    scale_factor_final = max(scale_x, scale_y)  # Используем максимальный коэффициент для вписывания

    # Масштабируем прямоугольник
    scaled_rect_width_px = int(rect_width_px * scale_factor_final)
    scaled_rect_height_px = int(rect_height_px * scale_factor_final)

    # Вычисляем новые координаты масштабированного прямоугольника
    scaled_rect_top_left = (center_x - scaled_rect_width_px // 2, center_y - scaled_rect_height_px // 2)
    scaled_rect_bottom_right = (center_x + scaled_rect_width_px // 2, center_y + scaled_rect_height_px // 2)

    # Рисование масштабированного прямоугольника
    cv2.rectangle(output_image, scaled_rect_top_left, scaled_rect_bottom_right, (0, 255, 0), 2)  # Зеленый цвет (BGR)

    # Вычисление координат точек A'', B'', C'', D'' для масштабированного прямоугольника
    scaled_rect_A_double_prime = (scaled_rect_top_left[0], scaled_rect_top_left[1])
    scaled_rect_B_double_prime = (scaled_rect_top_left[0], scaled_rect_bottom_right[1])
    scaled_rect_C_double_prime = (scaled_rect_bottom_right[0], scaled_rect_top_left[1])
    scaled_rect_D_double_prime = (scaled_rect_bottom_right[0], scaled_rect_bottom_right[1])

    # Сохранение координат масштабированного прямоугольника в словарь
    scaled_rect_double_prime_coords = {
        "A''": scaled_rect_A_double_prime,
        "B''": scaled_rect_B_double_prime,
        "C''": scaled_rect_C_double_prime,
        "D''": scaled_rect_D_double_prime
    }

    # Сохранение координат в файл
    with open("scaled_rect_double_prime_coords.txt", "w") as file:
        for point, coords in scaled_rect_double_prime_coords.items():
            file.write(f"{point}: {coords}\n")

    # Рисование точек A'', B'', C'', D'' на изображении
    cv2.circle(output_image, scaled_rect_A_double_prime, 5, (255, 0, 0), -1)  # Точка A'' (синий)
    cv2.putText(output_image, "A''", (scaled_rect_A_double_prime[0] + 10, scaled_rect_A_double_prime[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.circle(output_image, scaled_rect_B_double_prime, 5, (255, 0, 0), -1)  # Точка B'' (синий)
    cv2.putText(output_image, "B''", (scaled_rect_B_double_prime[0] + 10, scaled_rect_B_double_prime[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.circle(output_image, scaled_rect_C_double_prime, 5, (255, 0, 0), -1)  # Точка C'' (синий)
    cv2.putText(output_image, "C''", (scaled_rect_C_double_prime[0] + 10, scaled_rect_C_double_prime[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.circle(output_image, scaled_rect_D_double_prime, 5, (255, 0, 0), -1)  # Точка D'' (синий)
    cv2.putText(output_image, "D''", (scaled_rect_D_double_prime[0] + 10, scaled_rect_D_double_prime[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_output_image{ext}"
new_image_path = os.path.join('intermediate_results/', new_filename)

# Сохранение изображения с результатом
cv2.imwrite(new_image_path, output_image)

# Показ результата
cv2.imshow('Final Result (Combined)', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Координаты точек пересечения:")
for point, coords in intersections.items():
    print(f"{point}: {coords}")

print("Координаты точек A', B', C', D':")
for point, coords in rect_prime_coords.items():
    print(f"{point}: {coords}")

print("Координаты точек A'', B'', C'', D'':")
for point, coords in scaled_rect_double_prime_coords.items():
    print(f"{point}: {coords}")

# Функция для получения доминирующего цвета изображения
def get_dominant_color(image):
    # Преобразуем изображение в массив пикселей
    pixels = np.float32(image.reshape(-1, 3))

    # Определяем критерии для кластеризации
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.1)
    _, labels, palette = cv2.kmeans(pixels, 1, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Возвращаем доминирующий цвет
    dominant_color = palette[0].astype(np.uint8)
    return dominant_color

# Определяем доминирующий цвет на исходном изображении
dominant_color = get_dominant_color(image)

# Вычисление преобразования перспективы
# Исходные точки (A, B, C, D)
pts1 = np.float32([intersection_A, intersection_B, intersection_C, intersection_D])

# Целевые точки (A'', B'', C'', D'')
pts2 = np.float32([scaled_rect_A_double_prime, scaled_rect_B_double_prime, scaled_rect_C_double_prime, scaled_rect_D_double_prime])

# Вычисляем матрицу преобразования перспективы
M = cv2.getPerspectiveTransform(pts1, pts2)

# Определяем размеры выходного изображения
# Получаем координаты углов исходного изображения
h, w = image.shape[:2]
corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]])

# Применяем преобразование к углам, чтобы определить новые границы
transformed_corners = cv2.perspectiveTransform(np.array([corners]), M)[0]

# Вычисляем новые размеры изображения
x_min = int(np.min(transformed_corners[:, 0]))
x_max = int(np.max(transformed_corners[:, 0]))
y_min = int(np.min(transformed_corners[:, 1]))
y_max = int(np.max(transformed_corners[:, 1]))

new_width = x_max - x_min
new_height = y_max - y_min

# Создаем матрицу сдвига для корректировки обрезания
shift_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])

# Объединяем матрицу преобразования и сдвига
M_shifted = shift_matrix @ M

# Функция для определения доминирующего цвета
def get_dominant_color(image, k=1):
    """
    Возвращает доминирующий цвет изображения с использованием K-means.
    :param image: Изображение в формате BGR.
    :param k: Количество кластеров (по умолчанию 1 для доминирующего цвета).
    :return: Доминирующий цвет в формате BGR.
    """
    # Преобразуем изображение в массив пикселей
    pixels = image.reshape(-1, 3)

    # Применяем K-means для кластеризации
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)

    # Получаем доминирующий цвет (центр самого большого кластера)
    dominant_color = kmeans.cluster_centers_[0].astype(int)
    return dominant_color

# Создаем пустое изображение для результата
warped_image = cv2.warpPerspective(image, M_shifted, (new_width, new_height))

# Создаем маску для добавленных областей (пустых после преобразования)
mask = (warped_image == 0).all(axis=2).astype(np.uint8) * 255

# Находим контуры добавленных областей
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Определяем доминирующий цвет исходного изображения
dominant_color = get_dominant_color(image)

# Минимальный размер области для обработки (30 пикселей)
min_area = 40

# Обрабатываем только области с размером не менее 30 пикселей
for contour in contours:
    # Проверяем размер области
    if cv2.contourArea(contour) >= min_area:
        # Создаем маску для текущей области
        contour_mask = np.zeros_like(mask)
        cv2.drawContours(contour_mask, [contour], -1, 255, thickness=cv2.FILLED)

        # Расширяем маску области на 5 пикселей для захвата окантовки
        kernel = np.ones((5, 5), np.uint8)  # Ядро для расширения
        expanded_contour_mask = cv2.dilate(contour_mask, kernel, iterations=1)

        # Заливаем область и окантовку доминирующим цветом
        warped_image[expanded_contour_mask == 255] = dominant_color

        # Размываем края области
        # Создаем маску для границ (области, где добавленные области встречаются с исходным изображением)
        border_mask = cv2.Canny(contour_mask, 100, 200)  # Выделяем границы с помощью Canny
        border_mask = cv2.dilate(border_mask, None, iterations=2)  # Расширяем границы

        # Применяем размытие только к границам
        blurred = cv2.GaussianBlur(warped_image, (15, 15), 0)  # Размываем все изображение
        warped_image[border_mask == 255] = blurred[border_mask == 255]  # Применяем размытие только к границам


# Формирование нового имени файла
directory, filename = os.path.split(image_path)
name, ext = os.path.splitext(filename)
new_filename = f"{name}_warped_image{ext}"
new_image_path = os.path.join('Output_data/', new_filename)

# Сохранение результата
cv2.imwrite(new_image_path, warped_image)

# Показ результата
cv2.imshow('Warped Image with Dominant Fill and Blurred Edges', warped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Исправление перспективы завершено. Добавленные области (не менее 30 пикселей) залиты доминирующим цветом, края размыты. Результат сохранен в файл 'warped_output_dominant_blurred_edges.jpg'.")

Координаты точек пересечения:
A: (157, 47)
B: (123, 425)
C: (821, 36)
D: (882, 387)
Истинная ширина прямоугольника: 1.0 метров
Истинная высота прямоугольника: 0.49940876693192293 метров
Координаты точек пересечения:
A: (157, 47)
B: (123, 425)
C: (821, 36)
D: (882, 387)
Координаты точек A', B', C', D':
A': (400, 195)
B': (400, 243)
C': (500, 195)
D': (500, 243)
Координаты точек A'', B'', C'', D'':
A'': (54, 25)
B'': (54, 413)
C'': (846, 25)
D'': (846, 413)
Исправление перспективы завершено. Добавленные области (не менее 30 пикселей) залиты доминирующим цветом, края размыты. Результат сохранен в файл 'warped_output_dominant_blurred_edges.jpg'.
